In [ ]:
### Process model output and save ensemble data

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import matplotlib
import matplotlib.pyplot as plt
import cftime
import dask
import statsmodels.api as sm
import os
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
### import some analysis functions we wrote for this project
import sys ; sys.path.append("../") ; sys.path.append("../../") ; sys.path.append("../pyth/")
from ppe_analysis.analysis import *
from pyfunctions import *
from emulation_pyfunctions import *

ModuleNotFoundError: No module named 'ppe_analysis'

In [ ]:
# Setup your PBSCluster
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
ncores=1
nmem='10GB'
cluster = PBSCluster(
    cores=ncores, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
    project='P93300641', # Input your project ID here
    walltime='01:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)

# Scale up
cluster.scale(20)

# Setup your client
client = Client(cluster)

In [ ]:
client.cluster

In [ ]:
def write_metric(da,v,m,ln='',units='',years=''):
    f='./processed_data/'+v+'.nc'
    if glob.glob(f):
        dsout = xr.open_dataset('./processed_data/out.nc')
        if m in dsout:
            if dsout[m].equals(da)==0:
                raise Exception(m+" already exists in "+f+", and does not match input DataArray")
            else:
                update=False
        else:
            update=True
    else:
        dsout=xr.Dataset()
        update=True
    if update:
        dsout[m]=da
        dsout[m].attrs={'long_name':ln,'units':units,'years':years}
        if glob.glob(f):
            dsout.to_netcdf(f+'.tmp')
            os.system('mv '+f+'.tmp '+f)
        else:
            dsout.to_netcdf(f)

In [ ]:
### LHC ensemble

In [ ]:
# CLM-PPE
dvs=['TLAI']
htape='h1'
yr0=2003
yr1=2014
ds=get_exp('transient',dvs=dvs,tape=htape,yy=(yr0,yr1))

In [ ]:
t=slice(str(yr0),str(yr1))
da=ds.TLAI.sel(time=t)
la=xr.open_dataset('landarea_transient.nc').landarea_pft.sel(time=t)
lai_amax=amax(da).mean(dim='year').compute()
PPE_lai = pmean(lai_amax,la).mean(dim='time').compute()

In [ ]:
# save files
v='TLAI'
m='TLAI_pftavg_annMax_2003-2014'
ln='PFT average LAImax 2003'
years=str(yr0)+'-'+str(yr1)
units=ds[v].attrs['units']
write_metric(PPE_lai,v,m,ln=ln,units=units,years=years)

In [ ]:
### pftLAI experiment

In [ ]:
# load LHC LAI 
# LH move experiment to daniels campaign directory
ds_new_h1=get_exp('pftLAI',dvs=['TLAI'],tape='h1',yy=(2003,2014))

t=slice(str(2003),str(2014))
da_new = ds_new_h1.TLAI.sel(time=t)

In [ ]:
amax_lai=amax(da_new).mean(dim='year').compute()

In [ ]:
PPE_lai_new = pmean(amax_lai,la).mean(dim='time').compute()

In [ ]:
# save files
v='TLAI'
m='TLAI_pftavg_annMax_2003-2014'
ln='PFT average LAImax 2003'
years=str(yr0)+'-'+str(yr1)
units=ds_new_h1[v].attrs['units']
write_metric(PPE_lai_new,v,m,ln=ln,units=units,years=years)